# Randomized FS for n # of features:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import pickle

In [2]:
X = pd.read_csv('All Features.csv')
Y = pd.read_csv('out-0812-pdi.csv')
X = X.drop(['Ref','Buffer'],axis = 1)
X

,Hydrocarbon \nTail Length,Percent\nCholesterol,Actual\nTemperature,Flow \nrate,Osmolarity\n(mOsm/L),Particle Size\n(Online),PDI\n(online),ALogP,ALogp2,AMR,...,MW,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WPATH,WPOL,XLogP,Zagreb
0,14.0,25.0,19.88,160,268.88,197.20,0.050,-8.4122,70.765109,146.5297,...,677.499555,5.741522,89.510878,1.945889,28.578334,21.958447,11471,49,11.576,196
1,14.0,25.0,19.90,160,268.88,217.20,0.246,-8.4122,70.765109,146.5297,...,677.499555,5.741522,89.510878,1.945889,28.578334,21.958447,11471,49,11.576,196
2,14.0,25.0,20.28,160,268.88,228.10,0.136,-8.4122,70.765109,146.5297,...,677.499555,5.741522,89.510878,1.945889,28.578334,21.958447,11471,49,11.576,196
3,14.0,25.0,20.41,115,249.36,319.80,0.171,-8.4122,70.765109,146.5297,...,677.499555,5.741522,89.510878,1.945889,28.578334,21.958447,11471,49,11.576,196
4,14.0,25.0,20.47,115,249.36,301.80,0.196,-8.4122,70.765109,146.5297,...,677.499555,5.741522,89.510878,1.945889,28.578334,21.958447,11471,49,11.576,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,17.8,50.0,58.50,115,14.84,62.43,0.075,-10.1402,102.823656,163.9993,...,761.593455,5.599952,101.510875,1.952132,28.578440,21.958552,16998,55,14.990,220
945,17.8,50.0,59.50,160,16.00,51.71,0.033,-10.1402,102.823656,163.9993,...,761.593455,5.599952,101.510875,1.952132,28.578440,21.958552,16998,55,14.990,220
946,17.8,50.0,59.70,160,16.00,49.07,0.085,-10.1402,102.823656,163.9993,...,761.593455,5.599952,101.510875,1.952132,28.578440,21.958552,16998,55,14.990,220
947,17.8,50.0,59.90,160,16.00,48.24,0.092,-10.1402,102.823656,163.9993,...,761.593455,5.599952,101.510875,1.952132,28.578440,21.958552,16998,55,14.990,220


In [44]:
# Randomly select 10 features in each step, choose best feature based on model performance, and move on to the next
# feature, for a total of n features:
def mean_relative_error(y_test, pred):
    y_test_array = np.array(y_test)
    mre_error = abs(pred.flatten() - y_test_array)/abs(y_test_array)
    mre = np.sum(mre_error)/pred.size
    return mre

def model(X1, Y):
    x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.33, random_state=42)
    np.random.seed(42)
    tf.random.set_seed(42)

    # from tensorflow.keras.callbacks import ModelCheckpoint
    NN_model = Sequential()

    # The Input Layer :
    NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

    # The Hidden Layers :
    NN_model.add(Dense(64, kernel_initializer='normal',activation='linear'))
    NN_model.add(Dense(64, kernel_initializer='normal',activation='linear'))
    NN_model.add(Dense(64, kernel_initializer='normal',activation='linear'))
    NN_model.add(Dense(64, kernel_initializer='normal',activation='linear'))


    # The Output Layer :
    NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

    # Compile the network :
    NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae', 'mse'])
    #NN_model.summary()
    NN_model.fit(x_train, y_train, epochs=10)
    
    pred = NN_model.predict(x_test)
    
    res = mean_relative_error(y_test, pred)
    return res
    
def best_feat(df, num_feats , best_n_feats):
    # determins best local feature, and references previous features
    res = []
    for z in range(df.shape[1]):
        X1 = df[[df.columns[z]]]
        if num_feats != 0:
            prev_feats = X[best_n_feats]
            X_com = pd.concat([X1, prev_feats], axis = 1)
            acc = model(X_com,Y)
        else: 
            acc = model(X1,Y)
            X_com = []
        res.append(acc)
        best = min(res)
        best_ft = res.index(best)
    return best_ft

#def best_feats(df):
    # determine best new feature by 
    
def for_feat_sel(num_of_feats):
    df_new_ft = []
    while len(df_new_ft) < num_of_feats:
#     for i in range(100):
        # Randomly select 10 features to compare their performance
        X = X_new
        df = X.sample(n=25, replace = 'True', axis = 'columns', random_state = 15)
        num_feats = len(df_new_ft)
        ind_bf = best_feat(df, num_feats, df_new_ft)
        
        # Only add feature if it is not present in the best features list
        if df.columns[ind_bf] not in df_new_ft:
            df_new_ft.append(df.columns[ind_bf])
            print('Adding feature: ',df.columns[ind_bf])
  
        Xnew = X.drop([df.columns[ind_bf]],axis = 1)
            
    return df_new_ft
            
feats_50 = for_feat_sel(50)
    
with open('feature_sel_feats.pkl', 'wb') as f:
    pickle.dump(feats_50, f)


Epoch 1/10
20/20 [==============================] - 0s 1ms/step - loss: 0.1938 - mae: 0.1938 - mse: 6.1017
Epoch 2/10
20/20 [==============================] - 0s 1ms/step - loss: 0.1039 - mae: 0.1039 - mse: 1.9477
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.1265 - mae: 0.1265 - mse: 3.6130
Epoch 4/10
20/20 [==============================] - 0s 1ms/step - loss: 0.0734 - mae: 0.0734 - mse: 0.8297
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.1745 - mae: 0.1745 - mse: 6.3273
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.0950 - mae: 0.0950 - mse: 2.2257
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.1438 - mae: 0.1438 - mse: 4.8125
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2907 - mae: 0.2907 - mse: 12.2136
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2890 - mae: 0.2890 - mse: 12.5340
Epoch 10/10
20/20 [================

KeyboardInterrupt: 

In [42]:
#ind --> best_feat conv ind to a df --> feat --> column names --> df[column names] --> on all 50 feats
#X[df_new_ft]
feats_50
# X.drop([df.columns[ind_bf]],axis = 1)
#X.columns[ind_bf]

['Percent\nCholesterol',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',
 'Percent\nCholesterol',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',
 'Percent\nCholesterol',
 'Osmolarity\n(mOsm/L)',
 'Percent\nCholesterol',


In [22]:
X_new = pd.concat([X['Percent\nCholesterol'], X['Osmolarity\n(mOsm/L)']], axis = 1)

In [ ]:
X['ATS5s']